In [1]:
from collections import defaultdict

data_dir = 'data/SQuAD/'

split_titles = {"train": [], "dev": []}  # , "val": []
doc_qids = defaultdict(list)
qpa_examples = dict()

In [2]:
from html import unescape
import json
import os
from tqdm.notebook import tqdm

from utils.text_utils import norm_text

for split in ['dev', 'train']:
    with open(os.path.join(data_dir, f'{split}-v1.1.json')) as fin:
        docs = json.load(fin)['data']
        print(f'{len(docs)} docs in {split}')
        for doc in tqdm(docs):
            title = doc['title']
            norm_title = unescape(norm_text(title.replace('_', ' ')))
            split_titles[split].append(norm_title)
            for para in doc['paragraphs']:
                context = norm_text(para['context'])
                for qas in para['qas']:
                    q_id = qas['id']
                    question = norm_text(qas['question'])
                    answers = set(norm_text(ans['text']) for ans in qas['answers'])
                    doc_qids[norm_title].append(q_id)
                    qpa_examples[q_id] = (question, (norm_title, context), answers)
    print(f'{sum([len(doc_qids[t]) for t in split_titles[split]])} questions in {split}')
print(f'total {len(qpa_examples)} questions')

48 docs in dev



10570 questions in dev
442 docs in train



87599 questions in train
total 98169 questions


In [3]:
from elasticsearch import Elasticsearch


es = Elasticsearch(('10.60.0.59:9200',), timeout=120)
index_name='enwiki-20171001-paragraph-5'

qas_examples = dict()

In [5]:
for q_id, qpa in tqdm(qpa_examples.items()):
    if q_id in qas_examples:
        continue
    question, (norm_title, context), answers = qpa
    dsl = {
        "query": {
            "bool": {
                "must": [
                    {"term": {"title_unescaped.exact": norm_title}},
                    {"match": {"text": context}},
                    {"match": {"text": question}}
                ],
                "should": [{"match_phrase": {"text": ans}} for ans in answers],
                "must_not": {"term": {"para_idx": "-1"}},
                "minimum_should_match": 1
            }
        },
        "size": 3
    }
    hits = es.search(dsl, index_name)['hits']['hits']
    
    if len(hits) == 0:
#         print(q_id, json.dumps(dsl))
#         print(f"{q_id}: cannot find support passage")
        continue
    
    support_answers = []
    for i, hit in enumerate(hits):
        para = hit['_source']
        recalled_answers = [ans for ans in answers if ans.lower() in para['text'].lower()]
        if len(recalled_answers) > 0:
#             if i == 0 and para['title_unescaped'] != norm_title:
#                 print(f"{q_id}: find {recalled_answers} in 『{para['title_unescaped']}』 instead of 『{norm_title}』")
            support_answers.append((para['para_id'], para['title_unescaped'], recalled_answers))
#         elif i == 0:
#             print(f"{q_id}: cannot find {answers} in {para['para_id']}")
#             print(json.dumps(dsl))
    if len(support_answers) > 0:
        qas_examples[q_id] = (question, support_answers)
#     else:
#         print(f"{q_id}: cannot find support passage")
#         print(json.dumps(dsl))

In [25]:
import random

random.seed(0)

split_qids = {"train": [], "dev": [], "val": []}

for split in ['dev', 'train']:
    for t in split_titles[split]:
        if split == 'train' and random.random() < 48 / 442:
            partition = 'val'
        else:
            partition = split
        for q_id in doc_qids[t]:
            if q_id in qas_examples:
                split_qids[partition].append(q_id)
    print(f'{len(split_qids[partition])} questions remained in {split}')
print(f'{len(split_qids["val"])} questions remained in val')
print(f'total {len(qas_examples)} questions remained')

9530 questions remained in dev
68112 questions remained in train
8356 questions remained in val
total 85998 questions remained


In [27]:
for split, q_ids in split_qids.items():
    with open(os.path.join(f'data/squad-{split}-v1.1.tsv'), 'w') as fout:
        for q_id in q_ids:
            question, support_answers = qas_examples[q_id]
            assert len(support_answers) > 0
            fout.write(f"{q_id}\t{question}\t{repr(qpa_examples[q_id][2])}\t{repr([x[0] for x in support_answers])}\n")